In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import random
from pprint import pprint

#Load and Prepare Data

In [ ]:
df = pd.read_csv("IRIS.csv")
# df = df.drop("Id", axis=1)
df = df.rename(columns={"species": "label"})
df

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
# df=pd.read_csv("bank.csv")
# df

In [ ]:
# df=df.drop(['Id'], axis = 1)

In [ ]:
# df=data

#Train-Test-Split

In [ ]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [ ]:
# random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

#Helper Functions

In [ ]:
data = train_df.values
data[:5]

array([[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5.0, 3.6, 1.4, 0.2, 'Iris-setosa']], dtype=object)


#Data pure?

In [ ]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

#Classify

In [ ]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

#Potential splits?

In [ ]:
# def get_potential_splits(data):
#   potential_splits = {}
#   _, n_columns = data.shape
#   for column_index in range(n_columns - 1): # excluding the last column which is the label  


#     values = data[:, column_index]
#     unique_values = np.unique(values)
#     type_of_feature=feature_types[column_index]
#     if type_of_feature == "continuous":
#       potential_splits[column_index] = [] 
#       for index in range(len(unique_values)):
#         if index != 0:
#           current_value = unique_values[index]
#           previous_value = unique_values[index - 1]
#           potential_split = (current_value + previous_value) / 2
#           potential_splits[column_index].append(potential_split)
#     elif len(unique_values) > 1:
#       potential_split[column_index]=unique_values
#       print(potential_split)

#   # return potential_splits

In [ ]:
def get_potential_splits(data):
  potential_splits = {}
  _, n_columns = data.shape
  for column_index in range(n_columns - 1): # excluding the last column which is the label
      values = data[:, column_index] #show column one by one
      unique_values = np.unique(values) #find unique value in column
      
      # type_of_feature = feature_types[column_index] #show which type of feature 
      # if type_of_feature == "continuous":
      #     potential_splits[column_index] = [] #store data in dit
      #     for index in range(len(unique_values)):
      #         if index != 0:
      #             current_value = unique_values[index]
      #             previous_value = unique_values[index - 1]
      #             potential_split = (current_value + previous_value) / 2 #find median
      #             potential_splits[column_index].append(potential_split)
      
      # # feature is categorical
      # # (there need to be at least 2 unique values, otherwise in the
      # # split_data function data_below would contain all data points
      # # and data_above would be empty)
      # elif len(unique_values) > 1: #if data is catagorial than this part work
      potential_splits[column_index] = unique_values
  
  return potential_splits



In [ ]:
# get_potential_splits(train_data.values)

#Split Data

In [ ]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature=feature_types[split_column]
    if type_of_feature == "continuous":
      data_below = data[split_column_values <= split_value]
      data_above = data[split_column_values >  split_value]
    else:
      data_below = data[split_column_values == split_value]
      data_above = data[split_column_values !=  split_value]      
    
    return data_below, data_above

In [ ]:
# data_below,data_above=split_data(data.values,split_column=1,split_value="male")

In [ ]:
# data_above

In [ ]:
# np.unique(data_below[:,1])

#Lowest Overall Entropy?

In [ ]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [ ]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [ ]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

#Determine Type of Feature 

In [ ]:
def determine_type_of_feature(data):
  feature_type=[]
  n_unique_values_threshold=15

  for column in data.columns:
    unique_values=data[column].unique()#unique value in column 
    example_value=unique_values[0]#show data in 0 number row
    
    if (isinstance(example_value,str) or (len(unique_values) <= n_unique_values_threshold)):
      feature_type.append("categorical")
    else:
      feature_type.append("continuous")

  return feature_type

In [ ]:
determine_type_of_feature(df)

['continuous', 'continuous', 'continuous', 'continuous', 'categorical']

In [ ]:
feature_types=determine_type_of_feature(df)
i=0
for column in df.columns:
  print(column,"-",feature_types[i])
  i +=1

sepal_length - continuous
sepal_width - continuous
petal_length - continuous
petal_width - continuous
label - categorical


#Decision Tree Algorithm
Representation of the Decision Tree

In [ ]:
def decision_tree_algorithm(df, counter=0,min_samples=2,max_depth=3):
  
  # data preparations
  if counter == 0:
      global column_headers ,feature_types#its show column name
      column_headers=df.columns
      feature_types=determine_type_of_feature(df)
      data = df.values
  else:
      data = df           
  
  
  # base cases
  if check_purity(data) or (len(data) < min_samples) or (counter == max_depth):
      classification = classify_data(data)
      return classification

  
  # recursive part
  else:    
      counter += 1

      # helper functions 
      potential_splits = get_potential_splits(data)
      split_column, split_value = determine_best_split(data, potential_splits)
      data_below, data_above = split_data(data, split_column, split_value)

      # check for empty data
      if len(data_below) == 0 or len(data_above) == 0:
          classification = classify_data(data)
          return classification

      # instantiate sub-tree
      feature_name=column_headers[split_column]
      type_of_feature=feature_types[split_column]
      if type_of_feature == "continuous":
        question = "{} <= {}".format(feature_name, split_value) #if the column is continuous than it show this type of output
      else:
        question = "{} = {}".format(feature_name, split_value) #else show this type of output 
      sub_tree = {question: []}
      
      # find answers (recursion)
      yes_answer = decision_tree_algorithm(data_below, counter,min_samples,max_depth)
      no_answer = decision_tree_algorithm(data_above, counter,min_samples,max_depth)
      
      if yes_answer==no_answer:
        sub_tree=yes_answer
      else:
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
      
      return sub_tree

In [ ]:
tree = decision_tree_algorithm(train_df,max_depth=3)
pprint(tree)

{'petal_width <= 0.6': ['Iris-setosa',
                        {'petal_width <= 1.7': [{'petal_length <= 5.0': ['Iris-versicolor',
                                                                         'Iris-virginica']},
                                                'Iris-virginica']}]}


In [ ]:
# train_df.columns

#Classification

In [ ]:
test_df.shape

(20, 5)

In [ ]:
 example=test_df.iloc[1]
 example

sepal_length               6.0
sepal_width                2.2
petal_length               5.0
petal_width                1.5
label           Iris-virginica
Name: 119, dtype: object

In [ ]:
example["petal_width"] <=0.8

False

In [ ]:
def classify_example(example,tree):
  question=list(tree.keys())[0] #convert data type dict to list
  feature_name,comparison_operator,value=question.split() #split data
  #ask question=Yes

  if comparison_operator == "<=":
    if  example[feature_name] <= float(value):
      answer=tree[question][0]
    #ask question=No
    else:
      answer=tree[question][1]

  else:
    if  example[feature_name] == (value):
      answer=tree[question][0]
    #ask question=No
    else:
      answer=tree[question][1]
  #base case
  if not isinstance(answer,dict): #if there is list then its return 
    return answer
  else: #if there is dict then its return 
    residual_tree=answer
    return classify_example(example,answer)

In [ ]:
classify_example(example,tree)

'Iris-versicolor'

In [ ]:
test_df.shape[0]

20

In [ ]:
# for i in range(test_df.shape[0]):
#   example=test_df.iloc[i]
#   k=classify_example(example,tree)
#   c=k==test_df.label
# print(c)

#Accuracy

In [ ]:
df["classification"]=df.apply(classify_example,axis=1,args=(tree,))
df["classification"]

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: classification, Length: 150, dtype: object

In [ ]:
df["classification_correct"]=df.classification==df.label
df["classification_correct"].mean()

0.9733333333333334

In [ ]:
def calculate_accuracy(df,tree):
  df["classification"]=df.apply(classify_example,axis=1,args=(tree,))
  df["classification_correct"]=df.classification==df.label

  accuracy=df.classification_correct.mean()
  return accuracy

In [ ]:
calculate_accuracy(df,tree)

0.9733333333333334

#Titanic Data Set

prepare and load data

In [ ]:
# train_data=pd.read_csv("train.csv")
# train_data["label"]=train_data.Survived
# train_data=train_data.drop(["PassengerId","Survived","Name","Ticket","Cabin"],axis=1)

In [ ]:
# test_data["label"]=test_data.Survived
# test_data=test_data.drop(["PassengerId","Survived","Name","Ticket","Cabin"],axis=1)

In [ ]:
# train_data.info()

In [ ]:
# test_data.info()

In [ ]:
# data=pd.read_csv("bank.csv")
# data

In [ ]:
# data = data.rename(columns={"deposit": "label"})

In [ ]:
# train_data, test_data = train_test_split(data, test_size=20)

In [ ]:
# train_data

Handaling missing values

In [ ]:
# median_age=train_data.Age.median()
# mode_embarked=train_data.Embarked.mode()[0]
# train_data=train_data.fillna({"Age":median_age,"Embarked":mode_embarked})
# test_data=test_data.fillna({"Age":median_age,"Embarked":mode_embarked})

In [ ]:
# data.info()

In [ ]:
tree=decision_tree_algorithm(train_df,max_depth=4)

In [ ]:
accuracy=calculate_accuracy(df,tree)

In [ ]:
tree

{'petal_width <= 0.6': ['Iris-setosa',
  {'petal_width <= 1.7': [{'petal_length <= 5.0': ['Iris-versicolor',
      'Iris-virginica']},
    {'petal_length <= 4.8': [{'sepal_width <= 3.0': ['Iris-virginica',
        'Iris-versicolor']},
      'Iris-virginica']}]}]}

In [ ]:
accuracy

0.98

In [ ]:
class desition_tree:
  def __init__(self):
    pass


  def check_purity(data):
      
      label_column = data[:, -1]
      unique_classes = np.unique(label_column)

      if len(unique_classes) == 1:
          return True
      else:
          return False

  def classify_data(data):
      
      label_column = data[:, -1]
      unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

      index = counts_unique_classes.argmax()
      classification = unique_classes[index]
      
      return classification

  def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1): # excluding the last column which is the label
        values = data[:, column_index] #show column one by one
        unique_values = np.unique(values) #find unique value in column
        
        type_of_feature = feature_types[column_index] #show which type of feature 
        if type_of_feature == "continuous":
            potential_splits[column_index] = [] #store data in dit
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2 #find median
                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1: #if data is catagorial than this part work
            potential_splits[column_index] = unique_values
    
    return potential_splits


  def split_data(data, split_column, split_value):
      
      split_column_values = data[:, split_column]

      type_of_feature=feature_types[split_column]
      if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
      else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values !=  split_value]      
      
      return data_below, data_above


  def calculate_entropy(data):
      
      label_column = data[:, -1]
      _, counts = np.unique(label_column, return_counts=True)

      probabilities = counts / counts.sum()
      entropy = sum(probabilities * -np.log2(probabilities))
      
      return entropy

  def calculate_overall_entropy(data_below, data_above):
      n = len(data_below) + len(data_above)
      p_data_below = len(data_below) / n
      p_data_above = len(data_above) / n
      overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                        + p_data_above * calculate_entropy(data_above))
      
      return overall_entropy


  def determine_best_split(data, potential_splits):
      overall_entropy = 9999
      for column_index in potential_splits:
          for value in potential_splits[column_index]:
              data_below, data_above = split_data(data, split_column=column_index, split_value=value)
              current_overall_entropy = calculate_overall_entropy(data_below, data_above)

              if current_overall_entropy <= overall_entropy:
                  overall_entropy = current_overall_entropy
                  best_split_column = column_index
                  best_split_value = value
      return best_split_column, best_split_value


  def determine_type_of_feature(data):
    feature_type=[]
    n_unique_values_threshold=15
    for column in data.columns:
      unique_values=data[column].unique()#unique value in column 
      example_value=unique_values[0]#show data in 0 number row
      
      if (isinstance(example_value,str) or (len(unique_values) <= n_unique_values_threshold)):
        feature_type.append("categorical")
      else:
        feature_type.append("continuous")

    return feature_type


  def fit_data(df, counter=0,min_samples=2,max_depth=3):
    
    # data preparations
    if counter == 0:
        global column_headers ,feature_types#its show column name
        column_headers=df.columns
        feature_types=determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if check_purity(data) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # instantiate sub-tree
        feature_name=column_headers[split_column]
        type_of_feature=feature_types[split_column]
        if type_of_feature == "continuous":
          question = "{} <= {}".format(feature_name, split_value) #if the column is continuous than it show this type of output
        else:
          question = "{} = {}".format(feature_name, split_value) #else show this type of output 
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter,min_samples,max_depth)
        no_answer = decision_tree_algorithm(data_above, counter,min_samples,max_depth)
        
        if yes_answer==no_answer:
          sub_tree=yes_answer
        else:
          sub_tree[question].append(yes_answer)
          sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
obj=desition_tree()

In [ ]:
obj.fit_data(train_df)

ValueError: ignored